# 課題1

別途支給するテスト用画像(136枚)から、ナンバープレートを検出するプログラムを開発してください。

#### 目的

ナンバープレートを検出する学習モデルの生成

#### 手順

##### 1. テストデータの配置

画像(.jpg)とアノテーション(.txt)のデータセットを訓練用(train)と評価用(val)に分割して本ディレクトリに配置する。


```
datasets
├　images
│　├　train
│　│　└　画像(.jpg)
│　└　val
│　 　└　YOLO形式(.txt)
└　labels
 　├　train
 　│　└　画像(.jpg)
 　└　val
 　 　└　YOLO形式(.txt)
```

##### 2. 訓練実行

以下実行

重みファイルがyolov5/runs/train/license_plate_detector/weight/に生成される。

In [1]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5/yolov5s.pt --name license_plate_detector

train: weights=yolov5/yolov5s.pt, cfg=, data=dataset.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=license_plate_detector, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.11.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA RTX A4000, 16102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8,

# 課題2

検出したナンバープレートにBounding Boxを描画した画像を出力してください。

#### 目的

訓練済みモデルを使用したBounding Boxの算出

元画像にBounding Boxを描画

#### 手順

##### 0. 課題1を実行する

##### 1. テストデータの配置

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. Bounding Boxの計算と描画

直下セル実行

出力先のディレクトリ

```
tests
└　output
 　└　画像(.jpg)
```

In [2]:
import math
import torch
import cv2
import glob
import os

# 訓練済みモデル
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各画像への処理
# Bounding Boxの計算と描画
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    file_name = os.path.splitext(os.path.basename(data_path))[0]
    img = cv2.imread(data_path)
    result = model(img)
    for row in result.pandas().xyxy[0].itertuples():
        cv2.rectangle(
            img, 
            (math.floor(row.xmin), math.floor(row.ymin)), # Bounding Boxの左上座標
            (math.floor(row.xmax), math.floor(row.ymax)), # Bounding Boxの右下座標
            color=(255, 255, 0), # 青クリーム色の枠線
            thickness=2 # 枠線の太さ
        )
    cv2.imwrite("{0}/{1}.jpg".format(output_img_data_path, file_name), img)

YOLOv5 🚀 v7.0-378-g2f74455a Python-3.11.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA RTX A4000, 16102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.am

# 課題3

黄色のナンバープレート(軽自動車)の件数を出力してください。